In [1]:
from collections import Counter
from dateutil import parser
from datetime import timezone
from pprint import pprint

import ipywidgets as ipyw
import traitlets as trt
from IPython.display import display

import requests

import networkx as nx

import ipyelk
import ipyelk.nx
from ipyelk.diagram.elk_model import ElkLabel

In [2]:
try:
    import sysml_v2_api_client as sysml2
    from sysml_v2_api_client.rest import ApiException
except ImportError:
    !pip install git+https://github.com/Systems-Modeling/SysML-v2-API-Python-Client.git
    print(
        "Had to install the SysML v2 Python API Client.\n"
        "Restart the kernel and run the notebook again..."
    )

In [3]:
class SysML2LabeledPropertyGraph(trt.HasTraits):

    graph = trt.Instance(nx.MultiDiGraph, kw=dict())
    
    def _update(self, client, merge=False):
        if not merge:
            old_graph = self.graph
            self.graph = nx.MultiDiGraph()
            del old_graph
            
        elements_by_id = client.elements_by_id

        relationship_element_ids = {
            element_id
            for element_id, element in elements_by_id.items()
            if 'relatedElement' in element
        }
        non_relationship_element_ids = set(
            elements_by_id
        ).difference(relationship_element_ids)
        
        self.graph.add_nodes_from(
            {
                element_id: elements_by_id[element_id]
                for element_id in non_relationship_element_ids
            }.items()
        )
        relationships = [
            elements_by_id[id_]
            for id_ in relationship_element_ids
        ]
        self.graph.add_edges_from([
            [
                relation["relatedElement"][0]["@id"],  # source node (str id)
                relation["relatedElement"][1]["@id"],  # target node (str id)
                relation["@type"],                     # edge type (str name)
                relation,                              # edge data (dict)
            ]
            for relation in relationships
        ])
        
    def subgraph(
        self,
        *,
        add_ipyelk_data: bool = True,
        edges: (list, tuple) = None,
        edge_types: (list, tuple, str) = None,
    ):
        graph = self.graph
        subgraph = type(graph)()

        edges = edges or []

        edge_types = edge_types or []
        if isinstance(edge_types, str):
            edge_types = [edge_types]

        if edge_types:
            edges += [
                (source, target, data)
                for (source, target, type_), data in graph.edges.items()
                if type_ in edge_types
            ]

        if not edges:
            print(f"Could not find any edges of type: '{edge_types}'!")
            return subgraph

        nodes = {
            node_id: graph.nodes[node_id]
            for node_id in sum([
                [source, target]
                for (source, target, data) in edges
            ], [])  # sum(a_list, []) flattens a_list
        }

        for id_, node_data in nodes.items():
            node_data["id"] = id_
            type_label = [ElkLabel(
                id=f"""type_label_for_{id_}""",
                text=f"""«{node_data["@type"]}»""",
                properties={
                    "cssClasses": "node_type_label",
                },
            )] if "@type" in node_data else []
            node_data["labels"] = type_label + [node_data["name"]]

        subgraph.add_nodes_from(nodes.items())
        subgraph.add_edges_from(edges)

        return subgraph
    
    def make_diagram(self, graph: nx.Graph) -> ipyw.HBox:
        layouts = ipyelk.nx.XELKTypedLayout()

        elk_diagram = ipyelk.Elk(
            transformer=ipyelk.nx.XELK(
                source=(graph, None),
                label_key="labels",
                layouts=layouts.value,
            ),
            style={
                " text.elklabel.node_type_label": {
                    "font-style": "italic",
                }
            },
        )

        def _element_type_opt_change(change):
            elk_diagram.transformer.layouts = layouts.value
            elk_diagram.refresh()

        layouts.observe(_element_type_opt_change, "value")
        elk_diagram.layout.flex = "1"
        
        # Make the direction and label placement look better...
        self.set_layout_option(layouts, "Parents", "Direction", "UP")
        self.set_layout_option(layouts, "Label", "Node Label Placement", "H_CENTER V_TOP INSIDE")

        return ipyw.HBox([elk_diagram, layouts], layout=dict(height="60vh"))
    
    @staticmethod
    def set_layout_option(widget, category: str, option: str, value):
        """Set a layout option"""
        category_idxs = [
            int(idx)
            for idx, name in widget._titles.items()
            if name == category
        ]
        if len(category_idxs) != 1:
            raise ValueError(f"Found {len(category_idxs)} entries for '{category}'!")
        category_widget = widget.children[category_idxs[0]]

        option_idxs = [
            int(idx)
            for idx, name in category_widget._titles.items()
            if name == option
        ]
        if len(option_idxs) != 1:
            raise ValueError(f"Found {len(option_idxs)} entries for '{option}' under '{category}'!")

        category_widget.children[option_idxs[0]].value = value

In [4]:
class SysML2Client(ipyw.VBox):

    host_url = trt.Unicode(
        default_value="http://sysml2-sst.intercax.com"
    )
    host_port = trt.Integer(
        default_value=9000,
        min=1,
        max=65535,
    )
    
    page_size = trt.Integer(
        default_value=2000,
        min=10,
    )

    _api_configuration = trt.Instance(sysml2.Configuration)
    _commits_api = trt.Instance(sysml2.CommitApi)
    _elements_api = trt.Instance(sysml2.ElementApi)
    _projects_api = trt.Instance(sysml2.ProjectApi)

    projects = trt.Dict()
    elements_by_id = trt.Dict()
    elements_by_type = trt.Dict()
    
    lpg = trt.Instance(SysML2LabeledPropertyGraph, kw=dict())
    
    host_url_input = trt.Instance(ipyw.Text)
    host_port_input = trt.Instance(ipyw.IntText)
    project_selector = trt.Instance(ipyw.Dropdown)
    commit_selector = trt.Instance(ipyw.Dropdown)
    download_elements = trt.Instance(ipyw.Button)
    type_selector = trt.Instance(ipyw.SelectMultiple, kw=dict())
    
    @trt.validate("children")
    def _validate_children(self, proposal):
        children = proposal.value
        if children:
            return children
        return [
            ipyw.HTML("<h2>SysML2 Remote Service</h2>"),
            ipyw.HBox([self.host_url_input, self.host_port_input]),
            ipyw.HBox([
                ipyw.VBox([
                    self.project_selector,
                    self.commit_selector,
                ], layout=ipyw.Layout(width="80%", max_width="32rem")),
                self.download_elements,
            ]),
            ipyw.HTML("<h2>Element Types<h2>"),
            self.type_selector,
        ]

    @trt.default("_api_configuration")
    def _make_api_configuration(self):
        return sysml2.Configuration(host=self.host)

    @trt.default("_commits_api")
    def _make_commits_api(self):
        with sysml2.ApiClient(self._api_configuration) as client:
            api = sysml2.CommitApi(client)
        return api

    @trt.default("_elements_api")
    def _make_elements_api(self):
        with sysml2.ApiClient(self._api_configuration) as client:
            api = sysml2.ElementApi(client)
        return api

    @trt.default("_projects_api")
    def _make_projects_api(self):
        with sysml2.ApiClient(self._api_configuration) as client:
            api = sysml2.ProjectApi(client)
        return api

    @trt.default("projects")
    def _make_projects(self):
        projects = self._projects_api.get_projects()
        return {
            project.id: dict(
                created=parser.parse(
                    " ".join(project.name.split()[-6:])
                ).astimezone(timezone.utc),
                full_name=project.name,
                name=" ".join(project.name.split()[:-6]),
            )
            for project in projects
        }
    
    @trt.default("host_url_input")
    def _make_host_url_input(self):
        input_box = ipyw.Text(default_value=self.host_url)
        trt.link(
            (self, "host_url"),
            (input_box, "value"),
        )
        layout = input_box.layout
        layout.width = "80%"
        layout.max_width = "30rem"
        return input_box
    
    @trt.default("host_port_input")
    def _make_host_port_input(self):
        input_box = ipyw.IntText(
            default_value=self.host_port,
            min=1,
            max=65535,
        )
        trt.link(
            (self, "host_port"),
            (input_box, "value"),
        )
        layout = input_box.layout
        layout.width = "15%"
        layout.max_width = "6rem"
        return input_box
    
    @trt.default("project_selector")
    def _make_project_selector(self):
        selector = ipyw.Dropdown(
            descriptor="Projects",
            options=self._get_project_options(),
        )
        selector.observe(self._update_commit_options, "value")
        selector.layout.width = "99%"
        selector.layout.max_width = "30rem"
        return selector
    
    @trt.default("commit_selector")
    def _make_commit_selector(self):
        selector = ipyw.Dropdown(
            descriptor="Commits",
            options=self._get_commit_options(),
        )
        selector.observe(self._update_elements, "value")
        selector.layout.width = "99%"
        selector.layout.max_width = "30rem"
        return selector
    
    @trt.default("download_elements")
    def _make_download_elements_button(self):
        button = ipyw.Button(
            icon="cloud-download",
            tooltip="Fetch elements from remote host.",
        )
        button.on_click(self._download_elements)
        layout = button.layout
        layout.height, layout.width = "90%", "15%"
        layout.max_width = "6rem"
        return button

    @trt.observe("host_url", "host_port")
    def _update_api_configuration(self, *_):
        self._api_configuration = self._make_api_configuration()
    
    @trt.observe("_api_configuration")
    def _update_apis(self, *_):
        for api_type in ("commit", "element", "project"):
            api_attr = f"_{api_type}s_api"
            old_api = getattr(self, api_attr)
            api_maker = getattr(self, f"_make{api_attr}")
            setattr(self, api_attr, api_maker())
            del old_api
        self.project_selector.options = self._get_project_options()
        
    @property
    def host(self):
        return f"{self.host_url}:{self.host_port}"

    @property
    def selected_project_id(self):
        return self.project_selector.value

    @property
    def selected_commit_id(self):
        return self.commit_selector.value

    @property
    def elements_url(self):
        return (
            f"{self.host}/"
            f"projects/{self.selected_project_id}/"
            f"commits/{self.selected_commit_id}/"
            f"elements?page[size]={self.page_size}"
        )
    
    @property
    def selected_elements_by_type(self):
        element_ids = sum(map(list, self.type_selector.value), [])
        return [
            self.elements_by_id[id_]
            for id_ in element_ids
        ]

    def _update_commit_options(self, *_):
        self.commit_selector.options = self._get_commit_options()

    def _update_elements(self, *_):
        pass

    def _get_project_options(self):
        project_name_instances = Counter(
            project["name"]
            for project in self.projects.values()
        )

        return {
            data["name"] + (
                f""" ({data["created"].strftime("%Y-%m-%d %H:%M:%S")})"""
                if project_name_instances[data["name"]] > 1
                else ""
            ): id_
            for id_, data in sorted(
                self.projects.items(),
                key=lambda x: x[1]["name"],
            )
        }

    def _get_commit_options(self):
        # TODO: add more info about the commit when API provides it
        return [
            commit.id
            for commit in self._commits_api.get_commits_by_project(
                self.selected_project_id
            )
        ]

    def _download_elements(self, *_):
        response = requests.get(self.elements_url)
        if not response.ok:
            raise requests.HTTPError(
                f"Failed to retrieve elements from '{self.elements_url}', "
                f"reason: {response.reason}"
            )
        elements = response.json()
        self.elements_by_id = {
            element["@id"]: element
            for element in elements
        }
        self.elements_by_type = {
            type_: tuple([
                el["@id"]
                for el in elements
                if el["@type"] == type_
            ])
            for type_ in set(element["@type"] for element in elements)
        }
        self.lpg._update(client=self)

    def by_id(id_: str):
        return self.elements_by_id[id_]

    def name_by_id(id_: str):
        return self.by_id(id_).get("Name")
    
    @trt.observe("elements_by_type")
    def _updated_type_selector_options(self, *_):
        self.type_selector.options = {
            f"{type_} [{len(elements)}]": elements
            for type_, elements in sorted(self.elements_by_type.items())
        }

In [5]:
client = SysML2Client()
client

SysML2Client(children=(HTML(value='<h2>SysML2 Remote Service</h2>'), HBox(children=(Text(value='http://sysml2-…

In [6]:
client.project_selector.value = "d2922960-028d-4534-a8ee-476aa26d1d25"
client._download_elements()

In [7]:
[element["name"] for element in client.selected_elements_by_type]

[]

In [8]:
diagram = client.lpg.make_diagram(
    graph=client.lpg.subgraph(edge_types="Superclassing")
)
elk_app, *_ = diagram.children
diagram

In [10]:
# Select the root node in the diagram...
# ... or you can manually select one yourself
elk_app.selected = ["4082f336-ca78-4343-9776-303de7db3498"]

In [11]:
first_element_selected, *_ = elk_app.selected
client.elements_by_id[first_element_selected]

{'@context': {'@base': 'http://sysml2-sst.intercax.com:9000/projects/d2922960-028d-4534-a8ee-476aa26d1d25/commits/61cb843b-b6ad-4868-9fb8-29094e59827e/elements/',
  '@vocab': 'http://omg.org/ns/sysml#',
  'sysml': 'http://omg.org/ns/sysml#',
  'dcterms': 'http://purl.org/dc/terms/',
  'xsd': 'http://www.w3.org/2001/XMLSchema#',
  'aliasId': {'@type': 'xsd:string'},
  'documentation': {'@type': '@id'},
  'documentationComment': {'@type': '@id'},
  'endFeature': {'@type': '@id'},
  'feature': {'@type': '@id'},
  'featureMembership': {'@type': '@id'},
  'flowFeature': {'@type': '@id'},
  'humanId': {'@type': 'xsd:string'},
  'identifier': {'@type': 'dcterms:identifier'},
  'importedMembership': {'@type': '@id'},
  'inheritedFeature': {'@type': '@id'},
  'inheritedMembership': {'@type': '@id'},
  'input': {'@type': '@id'},
  'isAbstract': {'@type': 'xsd:boolean'},
  'isConjugated': {'@type': 'xsd:boolean'},
  'isSufficient': {'@type': 'xsd:boolean'},
  'isVariation': {'@type': 'xsd:boolean

# TODOs
1. Finish fleshing out the process in the `Kerbal Model.ipynb`
2. Modify the subgraph generator so it can take the value from the `Type Selector` directly
3. Improve the ipyelk diagram widget (may need to make improvements to `ipyelk`)
   * Add arrows
   * Add compartments
   * Fix layout
   * Add widget to see node details
4. Add RDF formulation